In [ ]:
pip install PyPDF2

In [ ]:
import PyPDF2
import pandas as pd
import regex 
import re

In [ ]:
pdfReader = PyPDF2.PdfReader('Pdf file to extract')
print(len(pdfReader.pages))

In [ ]:
pageObj = pdfReader.pages[3]#checked for 17
data=pageObj.extract_text()
data = data.replace('  ',',')
data=data.replace('  ',',')
data=data.replace("). ",',')
data = data.replace('\n',',')
print(data)

In [ ]:
##page.3 (page 4)##################
data=data.replace("Page 4,",'')
#####################

In [ ]:
df = pd.DataFrame([data1.split(',') for data1 in data.split('\n')])
df = df.dropna() 
df=df.reset_index(drop=True)
df=df.T
#df.drop(index=df.index[:44],inplace=True)
#df.drop(index=df.index[:48],inplace=True)####for pages.3
#df.drop(index=df.index[:8],inplace=True)####for pages.4
df=df.reset_index(drop=True)
df=df.rename(columns={ df.columns[0]: "A"})
df= df.apply(lambda x: x.replace({'PO Boxes':'', 'Scone':'','GPO Boxes':'','PO Bags':'','GPO Boxes & Bags':''}, regex=True))
df = df[~df['A'].isin([''])]
df = df.dropna() 
df=df.reset_index(drop=True)
for i, row in df.iterrows():
  if(len(df['A'][i].strip())<=1):
    df['A'][i]=''
for i, row in df.iterrows():
  if("–" in str(df['A'][i]) or "/" in str(df['A'][i]) or "&" in str(df['A'][i])):
    df['A'][i]=''


In [ ]:
#df['B']=df.A.str.extract('[A-Z a-z]+([0-9–]+)')
#####get only numbers####
df['B']='#'+df.A.str.extract('([0-9 –]+)')+'#'

for i, row in df.iterrows():
  if(str(df['B'][i])=='# #'):
    df['B'][i]=''
df['B']= df['B'].fillna('')
df['B'] = df['B'].replace('nan','')
df['col_m']=''
df['A']=df['A'].str.replace('([0-9 –]+)','##',regex=True)

df['A']=df['A'].str.replace('([a-z ]+)','%',regex=True)
for i, row in df.iterrows():
  df['col_m'][i]=str(df['A'][i]).replace('##',' '+str(df['B'][i]))
  #if(df['A'][i].startswith('##')):
  #  df['col_m'][i]=str(df['B'][i])+'###'+str(df['A'][i])
  #else:
  #  df['col_m'][i]=str(df['A'][i])+'###'+str(df['B'][i])
df_int1=pd.DataFrame(columns=['A'])
df_int1['A']=df['col_m'].str.split('#',1).explode()
df_int1['A']= df_int1['A'].fillna('')
df_int1 = df_int1.replace('nan','')
df_int1 = df_int1[~df_int1['A'].isin([''])]
df_int1 = df_int1[~df_int1['A'].isin([' '])]
df_int1=df_int1.reset_index(drop=True)
#######explode by % #######
df_int2=pd.DataFrame(columns=['A'])
df_int2['A']=df_int1['A'].str.split('%',1).explode()
df_int2['A']= df_int2['A'].fillna('')
df_int2 = df_int2.replace('nan','')
df_int2 = df_int2[~df_int2['A'].isin([''])]
df_int2 = df_int2[~df_int2['A'].isin([' '])]
df_int2=df_int2.reset_index(drop=True)
################## remove single length ###########
for i, row in df_int2.iterrows():
  if(len(str(df_int2['A'][i]).strip())<=1):
    df_int2['A'][i]=''
  elif ("%" in str(df_int2['A'][i]) and "#" in str(df_int2['A'][i])):
    df_int2['A'][i]=''
######## replace % with space ########
df_int2['A'] = df_int2['A'].replace("%",'')

df_int2['A']= df_int2['A'].fillna('')
df_int2 = df_int2.replace('nan','')
df_int2 = df_int2.replace('##','')
df_int2 = df_int2[~df_int2['A'].isin([''])]
df_int2 = df_int2[~df_int2['A'].isin([' '])]
df_int2=df_int2.reset_index(drop=True)
########## replace ## with space #########
for i, row in df_int2.iterrows():
  df_int2['A'][i] = df_int2['A'][i].replace('##','')
############ second round of split##########
df_int3=pd.DataFrame(columns=['A'])
df_int3['A']=df_int2['A'].str.split('#',1).explode()
for i, row in df_int3.iterrows():
  if "%" in str(df_int3['A'][i]):
    df_int3['A'][i]=''
df_int3['A']= df_int3['A'].fillna('')
df_int3 = df_int3.replace('nan','')
df_int3 = df_int3[~df_int3['A'].isin([''])]
df_int3 = df_int3[~df_int3['A'].isin([' '])]
df_int3=df_int3.reset_index(drop=True)
df_int3['A'] = df_int3['A'].str.rstrip()
for i, row in df_int3.iterrows():
  if(i<len(df_int3)-1):
    temp=''
    temp=df_int3['A'][i+1]
    temp2=df_int3['A'][i]
    res1 = "".join(re.findall("[a-zA-Z]+", temp))
    res2="".join(re.findall("[a-zA-Z]+", temp2))
    if(len(res1)>3 and len(res2)>3):
      df_int3['A'][i]=df_int3['A'][i]+df_int3['A'][i+1]
      df_int3['A'][i+1]='%%%'
for i, row in df_int3.iterrows():
  df_int3['A'][i] = df_int3['A'][i].replace('%%%','')
for i, row in df_int3.iterrows():
  if(df_int3['A'][i]=="MC"):
    df_int3['A'][i] = ''
for i, row in df_int3.iterrows():
  if(df_int3['A'][i]=="V"):
    df_int3['A'][i] = ''
df_int3['A']= df_int3['A'].fillna('')
df_int3 = df_int3.replace('nan','')
df_int3 = df_int3[~df_int3['A'].isin([''])]
df_int3 = df_int3[~df_int3['A'].isin([' '])]
df_int3=df_int3.reset_index(drop=True)

In [ ]:
df_ff=df_int3.set_index([df_int3.index//3, df_int3.index%3+1])['A'].unstack().add_prefix('col')
df_ff

In [ ]:
df

In [ ]:
df_ff.to_excel("/content/data_sheetcleaned_3page4.xlsx", sheet_name='3')  

In [ ]:
df['B']=df.A.str.extract('([0-9 –]+)') #####get only number####
#df['C']=df.A.str.extract('([A-Za-z -]+)')
df['col_m']=''
df['C']=''
pattern = r'[0-9]'
for i, row in df.iterrows():
  df['C'][i]=re.sub(pattern,'%', str(df['A'][i]))
for i, row in df.iterrows():
  if "%%%%" in str(df['C'][i]):
    df['col_m'][i]=''
for i, row in df.iterrows():
  df['col_m'][i]=str(df['C'][i])+'####'+str(df['B'][i])

df_t=pd.DataFrame(columns=['A'])
df_t['A']=df['col_m'].str.split('####',1).explode()
df.drop(columns=['B', 'col_m'])
df_t['A']= df_t['A'].fillna('')
df_t = df_t[~df_t['A'].isin([''])]
df_t = df_t[~df_t['A'].isin([' '])]
df_t=df_t.reset_index(drop=True)
#####################################
df_t['B']=df_t.A.str.extract('[0-9]+(.*)')
df_t['B']= df_t['B'].fillna('')
for i, row in df_t.iterrows():
  df_t['A'][i]=str(df_t['A'][i])+'####'+str(df_t['B'][i])
df_tin=pd.DataFrame(columns=['A'])
df_tin['A']=df_t['A'].str.split('####',1).explode()
df_tin['A']= df_tin['A'].fillna('')
df_tin = df_tin.replace('nan','')
df_tin = df_tin[~df_tin['A'].isin([''])]
df_tin = df_tin[~df_tin['A'].isin([' '])]
df_tin=df_tin.reset_index(drop=True)
df_t=df_tin

In [ ]:
#df_t['B']=df_t.A.str.extract('([A-Z -]+)') ##get all non digits
df_t['B']=df_t.A.str.extract('([A-Z _]+$)') ##get all non digits

for i, row in df_t.iterrows():
  if(len(str(df_t['B'][i]).strip())<=1):
    df_t['B'][i]=''
df_t['C']=df_t.A.str.extract('([0-9 –]+)') ##get all digits
for i, row in df_t.iterrows():
  if(len(str(df_t['C'][i]).strip())<=1):
    df_t['C'][i]=''

df_t1=pd.DataFrame(columns=['A'])
df_t['T']=''
for i, row in df_t.iterrows():
  if "–" in str(df_t['C'][i]):
    df_t['C'][i]=''

for i, row in df_t.iterrows():
  df_t['T'][i]=str(df_t['C'][i])+'##'+str(df_t['B'][i])
  if(len(df_t['T'][i].strip())<=1):
    df_t['T'][i]=''

df_t=df_t.reset_index(drop=True)
df_t1=pd.DataFrame(columns=['A'])
df_t1['A']=df_t['T'].str.split('##',1).explode()
for i, row in df_t1.iterrows():
  if(len(df_t1['A'][i])<=1):
    df_t1['A'][i]=''
df_t1=df_t1.reset_index(drop=True)

In [ ]:
df_t1 = df_t1.replace('nan','')
df_t1 = df_t1[~df_t1['A'].isin([''])]
df_t1 = df_t1[~df_t1['A'].isin([' '])]
df_t1=df_t1.reset_index(drop=True)

In [ ]:
df_t1

In [ ]:
df_t1['B']=df_t1.A.str.extract('([A-Z -]+)')
df_t1['C']=''
for i, row in df_t1.iterrows():
  if(len(str(df_t1['B'][i]).strip())<=3):
    df_t1['B'][i]=''
df_t1['B']= df_t1['B'].fillna('')
for i, row in df_t1.iterrows():
  if(i<(df_t1.A.count()-1)):
    if(df_t1['B'][i+1]!=''):
      df_t1['C'][i]=df_t1['B'][i]+'##'+df_t1['B'][i+1]
for i, row in df_t1.iterrows():
  if(i<(df_t1.A.count()-1)):
    if(df_t1['C'][i+1]!=''):
      df_t1['C'][i]=''
for i, row in df_t1.iterrows():
  if(i<(df_t1.A.count()-1)):
    if(df_t1['B'][i]!='' and df_t1['C'][i]!=''):
      df_t1['A'][i]=df_t1['C'][i]
      df_t1['A'][i+1]=''

In [ ]:
df_cleaned

In [ ]:
df_cleaned=pd.DataFrame(columns=['A1'])
df_cleaned['A1']=df_t1['A']
df_cleaned = df_cleaned[~df_cleaned['A1'].isin([''])]
df_cleaned = df_cleaned[~df_cleaned['A1'].isin([' '])]
df_cleaned=df_cleaned.reset_index(drop=True)
for i, row in df_cleaned.iterrows():
  if(i<(df_cleaned.A1.count()-1)):
    if(str(df_cleaned['A1'][i]).strip()==str(df_cleaned['A1'][i+1]).strip()):
      df_cleaned['A1'][i+1]=''
df_cleaned = df_cleaned[~df_cleaned['A1'].isin([''])]
df_cleaned = df_cleaned[~df_cleaned['A1'].isin([' '])]
df_cleaned=df_cleaned.reset_index(drop=True)
df_cleaned = df_cleaned.replace('##',' ')
#df_cleaned

In [ ]:
df_ff=df_cleaned.set_index([df_cleaned.index//3, df_cleaned.index%3+1])['A1'].unstack().add_prefix('col')

In [ ]:
df_ff

In [ ]:
df_ff.to_excel("/content/data_sheetcleaned_17page18.xlsx", sheet_name='3')  